In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm, ttest_ind

## Задача 1. Оценить необходимый размер групп
  
Допустим, мы хотим провести эксперимент, в который попадают клиенты, совершившие покупку во время эксперимента.

Метрика — средняя выручка с пользователя за время эксперимента;
Продолжительность — одна неделя;
Уровень значимости — 0.05;
Допустимая вероятность ошибки II рода — 0.1;
Ожидаемый эффект — 20 рублей.
Оцените необходимый размер групп по данным о покупках за неделю с 21 по 28 февраля. Обратим внимание, что в выборку попадают события из полуинтервала `[datetime(2022, 2, 21), datetime(2022, 2, 28))`.

В качестве ответа введите необходимый размер групп, округлённый до целого числа десятков `round(x, -1)`.

In [2]:
sales = pd.read_csv('2022-04-01T12_df_sales.csv')

In [3]:
def get_sample_size_abs(epsilon, std, alpha=0.05, beta=0.2):
    t_alpha = norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = norm.ppf(1 - beta, loc=0, scale=1)
    z_scores_sum_squared = (t_alpha + t_beta) ** 2
    sample_size = int(
        np.ceil(
            z_scores_sum_squared * (2 * std ** 2) / (epsilon ** 2)
        )
    )
    return sample_size

In [4]:
def get_sample_size_arb(mu, std, eff=1.01, alpha=0.05, beta=0.2):
    epsilon = (eff - 1) * mu

    return get_sample_size_abs(epsilon, std=std, alpha=alpha, beta=beta)

In [5]:
sales['date'] = pd.to_datetime(sales['date'])

In [6]:
start_date = pd.to_datetime('2022.02.21')
end_date = pd.to_datetime('2022.02.28')

In [7]:
ab_sales = sales[(sales['date'] >= start_date) & (sales['date'] < end_date)]

In [8]:
mu = ab_sales.groupby('user_id')['price'].sum().mean()
effect_size = (mu+20)/mu
std = ab_sales.groupby('user_id')['price'].sum().std()
alpha = 0.05
beta = 0.1

In [9]:
sample_size = get_sample_size_arb(mu, std, eff=effect_size, alpha=alpha, beta=beta)

In [10]:
n_users = round(sample_size, -1)
print(f'Размер выборки равен = {n_users}')

Размер выборки равен = 34570


## Задача 2. MDE

В прошлом задании получилось, что необходимый размер групп больше имеющихся данных за одну неделю. Какой минимальный эффект мы можем отловить с теми же вероятностями ошибок на данных
с 21 по 28 февраля?

Ответ округлите до целого значения.

In [11]:
def get_minimal_determinable_effect(std, sample_size, alpha=0.05, beta=0.2):
    t_alpha = norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = norm.ppf(1 - beta, loc=0, scale=1)
    disp_sum_sqrt = (2 * (std ** 2)) ** 0.5
    mde = (t_alpha + t_beta) * disp_sum_sqrt / np.sqrt(sample_size)
    return mde

Делим количество человек из нашего датафрейма на 2, поскольку у нас две выборки

In [12]:
sample_size = ab_sales.groupby('user_id')['price'].sum().shape[0]/2

In [13]:
mde = round(get_minimal_determinable_effect(std, sample_size, alpha=alpha, beta=beta))

In [14]:
print(f'mde = {np.round(mde)}')

mde = 33
